<a href="https://colab.research.google.com/drive/1ku4I9ViRNtwbh4zRTYMWLboPbDY6bZYG"
 target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepared by Basharat Hussain on 21/8/2022
This tutorial is modified version from machinelearningmastery.com

**Time series forecasting problems**

*   How to develop LSTM models for univariate time series forecasting.
*   How to develop LSTM models for multivariate time series forecasting.
*   How to develop LSTM models for multi-step time series forecasting.

**Prepration Steps -- access Google Drive and Mount Dataset**
 

This tutorial is divided into four parts; they are:

1. Univariate LSTM Models
    * Data Preparation
    * Vanilla LSTM
    * Stacked LSTM
    * Bidirectional LSTM
    * CNN LSTM
    * ConvLSTM
2. Multivariate LSTM Models
    * Multiple Input Series.
    * Multiple Parallel Series.
3. Multi-Step LSTM Models
    * Data Preparation
    * Vector Output Model
    * Encoder-Decoder Model
4. Multivariate Multi-Step LSTM Models
    * Multiple Input Multi-Step Output.
    * Multiple Parallel Input and Multi-Step Output.

# Univariate LSTM Models

**1.1 Data Preparation**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dataset_path = "/content/drive/My Drive/Google-CoLab/Seminar_ML_2022_CAMSATS/02. LSTM Models for Time Series Forecasting/pima-indians-diabetes.data.csv"
dataset_path

Mounted at /content/drive


'/content/drive/My Drive/Google-CoLab/Seminar_ML_2022_CAMSATS/02. LSTM Models for Time Series Forecasting/pima-indians-diabetes.data.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Google-CoLab/Seminar_ML_2022_CAMSATS/02. LSTM Models for Time Series Forecasting/pima-indians-diabetes.data.csv'

**1. Load Data**

define input sequence

raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
 
We divide the sequence into multiple input/output patterns called samples.

X,  y

10, 20, 30	>	   40

20, 30, 40	>	50

30, 40, 50	>	60

In [ ]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3

In [10]:
# first neural network with keras tutorial
# univariate data preparation
from numpy import array

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM




1.1 Split input sequence into array of Samples

Make a split function to convert [10, 20, 30, 40, 50, 60, 70, 80, 90] into sliding window samples 

In [11]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

Actual call of above function on inout sequence

In [12]:
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [13]:
print(X.shape)
print (y.shape)
print(X)

(6, 3)
(6,)
[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]


In [17]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1  # univariat ?
X = X.reshape((X.shape[0], X.shape[1], n_features))
for i in range(len(X)):
	print(X[i])

[[10]
 [20]
 [30]]
[[20]
 [30]
 [40]]
[[30]
 [40]
 [50]]
[[40]
 [50]
 [60]]
[[50]
 [60]
 [70]]
[[60]
 [70]
 [80]]


**2. Define Keras Model (Vanilla LSTM)**

In [18]:
# define the keras model
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))


**3. Compile Keras Model**



In [19]:
# compile the keras model
model.compile(optimizer='adam', loss='mse')

you can print summary of the compiled Model


In [20]:
# Print a summary of the Keras model:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


**4. Fit Keras Model**

Training occurs over epochs, and each epoch is split into batches.

**Epoch**: One pass through all of the rows in the training dataset

**Batch**: One or more samples considered by the model within an epoch before weights are updated

In [24]:
# fit the keras model on the dataset
model.fit(X, y, epochs=200, verbose=0)

**5. Make Predictions**

“After I train my model, how can I use it to make predictions on new data?”

In [25]:
from termcolor import colored
print(colored("hello red world **BOLD TEXT**", 'red'))

hello red world **BOLD TEXT**


In [33]:
# make probability predictions with the model
x_input_new = array([70, 80, 90])
y_input_new = 100

x_input_new = x_input_new.reshape((1, n_steps, n_features))
yhat = model.predict(x_input_new, verbose=0)
print(yhat)

print('%s => %f (expected %d)' % (x_input_new.tolist(), yhat, y_input_new))

[[100.59877]]
[[[70], [80], [90]]] => 100.598770 (expected 100)


# 1.2 Stacked LSTM

In [34]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))


model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.701935]]


# 1.3 Bidirectional LSTM

In [36]:
from tensorflow.keras.layers import Bidirectional

# define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[101.65765]]


# 1.4 CNN LSTM


[samples, subsequences, timesteps, features]

In [45]:
# univariate cnn lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)

print(X.shape)
print (y.shape)
print (X)
print (y)

# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

print(X.shape)
print (y.shape)
print (X)
print (y)

# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)
# demonstrate prediction
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
print (x_input)
yhat = model.predict(x_input, verbose=0)
print(yhat)

(5, 4)
(5,)
[[10 20 30 40]
 [20 30 40 50]
 [30 40 50 60]
 [40 50 60 70]
 [50 60 70 80]]
[50 60 70 80 90]
(5, 2, 2, 1)
(5,)
[[[[10]
   [20]]

  [[30]
   [40]]]


 [[[20]
   [30]]

  [[40]
   [50]]]


 [[[30]
   [40]]

  [[50]
   [60]]]


 [[[40]
   [50]]

  [[60]
   [70]]]


 [[[50]
   [60]]

  [[70]
   [80]]]]
[50 60 70 80 90]
[[[[60]
   [70]]

  [[80]
   [90]]]]
[[100.889435]]


# 1.5 ConvLSTM

[samples, timesteps, rows, columns, features]

In [38]:
# univariate convlstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)
# demonstrate prediction
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.26338]]
